In [4]:
### Lets start by reading a csv of coordinates

import os

directory= str(os.environ.get('HOME')) + '/Documents/my_projects/pythonGIS/data_package_beg/'
fileName = directory + "Points_training.csv"

### reading file line by line

f = open(fileName, 'r') 
for line in f: 
    print (line)
f.close()

X,Y,id

620053.849724248,5608269.55076969,1

620869.406785648,5608625.54789967,2

622371.067406639,5608735.5833762,3

622332.231356096,5608204.82401878,4

620429.264879496,5607220.97740503,5

619419.527565381,5607576.97453501,6

620319.229402957,5609512.30438706,7

621193.040540172,5609544.66776251,8

622688.228486073,5609518.77706215,9

623387.277395844,5608133.62459279,10



Now we can start using the ogr and gdal libraries. 
<br></br>
writing a point shapefile

In [5]:
#https://pcjericks.github.io/py-gdalogr-cookbook/geometry.html#create-a-point
from osgeo import ogr, osr

#fileName = r"D:\Cardiff\day1\data_package_beg\Points_training.csv"

### reading file line by line
lsData = []
f = open(fileName, 'r') 
for line in f: 
    data = line.split(',') ### we need to split the data based on comma!
    lsData.append(data)
f.close()

### So we have our data!! now create the shapefile

# 1. set up the shapefile driver
driver = ogr.GetDriverByName("ESRI Shapefile")

# 2.create the file
data_source = driver.CreateDataSource(directory + "points.shp")

# 3. set the spatial reference in this case UTM 30N
srs = osr.SpatialReference()
srs.ImportFromEPSG(32630) ### think about this a variable - build a function **

# 4. create the layer
layer = data_source.CreateLayer("points", srs, ogr.wkbPoint) ## look at ogr cook book

# 5. Add the fields to the shapefile
layer.CreateField(ogr.FieldDefn("Y", ogr.OFTReal)) ## data types
layer.CreateField(ogr.FieldDefn("X", ogr.OFTReal))

# 6. set the geometry as a POINT!
point = ogr.Geometry(ogr.wkbPoint)
# loop through the data
for i in range(1,len(lsData)):
    #create the feature
    feature = ogr.Feature(layer.GetLayerDefn())
    feature.SetField("Y", lsData[i][1]) ## i is iterator, 1 is position.. storing y,x not x,y
    feature.SetField("X", lsData[i][0])
    # add the point
    point.AddPoint(float(lsData[i][0]), float(lsData[i][1]), i)
    # set the wkt (well known text)
    wkt = point.ExportToWkt() ## print this if not sure..
    print (wkt)
    # Create the point from the Well Known Txt
    point = ogr.CreateGeometryFromWkt(wkt)

    # Set the feature geometry using the point
    feature.SetGeometry(point)
    # Create the feature in the layer (shapefile)
    layer.CreateFeature(feature)
    # Destroy the feature to free resources
    feature.Destroy()

# 7. Destroy the data source to free resources
data_source.Destroy() ## destroy can be a scary term, it would be better if it was called something else but... nothing we can do (well open source?)

POINT (620053.849724248 5608269.55076969 1)
POINT (620869.406785648 5608625.54789967 2)
POINT (622371.067406639 5608735.5833762 3)
POINT (622332.231356096 5608204.82401878 4)
POINT (620429.264879496 5607220.97740503 5)
POINT (619419.527565381 5607576.97453501 6)
POINT (620319.229402957 5609512.30438706 7)
POINT (621193.040540172 5609544.66776251 8)
POINT (622688.228486073 5609518.77706215 9)
POINT (623387.277395844 5608133.62459279 10)


In [6]:
### Reading a shapefile and printing its spatial reference
# Import the necessary modules
from  osgeo import ogr, osr

driver = ogr.GetDriverByName('ESRI Shapefile')
shp = driver.Open(directory + 'points.shp')

# Get Projection from layer
layer = shp.GetLayer()
spatialRef = layer.GetSpatialRef()
print (spatialRef)
print ("")
### get the number of features in a shapefile
featureCount = layer.GetFeatureCount()
print ("feature count:", featureCount)
print ("")
## get all the values in X field
print ("printing all the x values")
for feature in layer:
    print (feature.GetField("X"))

PROJCS["WGS 84 / UTM zone 30N",
    GEOGCS["WGS 84",
        DATUM["WGS_1984",
            SPHEROID["WGS 84",6378137,298.257223563,
                AUTHORITY["EPSG","7030"]],
            AUTHORITY["EPSG","6326"]],
        PRIMEM["Greenwich",0,
            AUTHORITY["EPSG","8901"]],
        UNIT["degree",0.0174532925199433,
            AUTHORITY["EPSG","9122"]],
        AUTHORITY["EPSG","4326"]],
    PROJECTION["Transverse_Mercator"],
    PARAMETER["latitude_of_origin",0],
    PARAMETER["central_meridian",-3],
    PARAMETER["scale_factor",0.9996],
    PARAMETER["false_easting",500000],
    PARAMETER["false_northing",0],
    UNIT["metre",1,
        AUTHORITY["EPSG","9001"]],
    AXIS["Easting",EAST],
    AXIS["Northing",NORTH],
    AUTHORITY["EPSG","32630"]]

feature count: 10

printing all the x values
620053.849724248
620869.406785648
622371.067406639
622332.231356096
620429.264879496
619419.527565381
620319.229402957
621193.040540172
622688.228486073
623387.277395844
